# Interfaces

We now continue our deep dive into software development by introducing a concept that is both abstract and foundational to modern software design: **interfaces**.

> *Definition*: **Interfaces** are a way of defining how different components of your software interact. More specifically, they describe a blueprint for what a component must do, without specifying *how* it must do it. In this sense, interfaces enforce a consistent structure and contract between parts of your code, enabling modularity, flexibility, and easier collaboration.

Although interfaces are not exclusive to object-oriented programming, we’ll introduce them within the context of your newfound appreciation for classes. That said, the ideas that follow&mdash;while illustrated with concrete examples&mdash;may still feel abstract or even a little inaccessible at first. This is completely normal. Understanding interfaces is essential for anyone with a long-term vision of building robust, scalable software, and the concepts often "click" only after you’ve seen them from multiple angles.

Revisit this section often, experiment with interfaces in your own code, and gradually you’ll begin to appreciate the remarkable power they wield.

## Interface definition

Recall that in the previous lecture ("Object-oriented programming"), we defined a `Dog` class with some attributes and methods. Now, imagine that another part of your codebase involves animals interacting with each other&mdash;for example, by talking. This part of the program doesn't care whether it's dealing with a `Dog`, a `Cat`, or some other animal. All it needs is for the object to implement a method like `say_hello`.

If both `Dog` and `Cat` implement `say_hello`, then from the perspective of the code that handles communication, they are interchangeable&mdash;it can treat them both as "talking animals". This is where an *interface* comes in: it defines a required method (in this case, `say_hello`) that any class must implement to be used in this way.

By focusing on behaviour rather than implementation details, interfaces allow you to design code that is more general, reusable, and easier to extend. An interface can be defined as follows:

In [ ]:
from typing import Protocol


class TalkingAnimalProtocol(Protocol):
    # Protocols don't care how an attribute is implemented, only that it exists
    name: str

    def say_hello(self, friend: str) -> str: ...

As simple as that!  Remember that crucially **an interface defines a contract for behaviour without implementing it.** It specifies *method signatures* that any implementing class must fulfill, but it includes no logic itself. So, you might be wondering, how do we actually use this interface in practice? Below is a simple example featuring some friendly, communicative animals. Each class implements a `say_hello` method:

In [ ]:
class Crocodile:
    """A chatty crocodile

    Args:
        name: Name

    Attributes:
        name: Name
    """

    def __init__(self, name: str):
        self.name: str = name

    def say_hello(self, friend: str) -> str:
        """Say hello to a friend

        Args:
            Name of the friend

        Returns:
            A hello message
        """
        return f"{self.name} the crocodile says: Snap! Hello, {friend}!"


# Docstrings are omitted for brevity but would follow the same pattern as shown above.
class Parrot:
    def __init__(self, name: str):
        self.name: str = name

    def say_hello(self, friend: str) -> str:
        return f"{self.name} the parrot says: Squawk! Hello, {friend}!"

Next, we instantiate our animal classes to create individual animals:

In [ ]:
crocodile = Crocodile("Chomper")
parrot = Parrot("Polly")

We now define a `greet` function and use a type hint to indicate that `TalkingAnimal`s are expected as arguments:

In [ ]:
# Define a greet function using the protocol as a type hint
def greet(animal1: TalkingAnimalProtocol, animal2: TalkingAnimalProtocol) -> None:
    """Animal greeting

    Args:
        animal1: A talking animal
        animal2: A second talking animal
    """
    print(animal1.say_hello(animal2.name))

Finally, we can let our two animal friends have a chat:

In [ ]:
greet(crocodile, parrot)

So far, you might be thinking this seems like a lot of effort! Especially in a language like Python, where type hints are not enforced by the interpreter. This means that without additional tooling&mdash;typically provided by an IDE&mdash;you won’t get immediate feedback if something goes wrong. Instead, errors (such as trying to call a `speak` method on an object that doesn’t have one) will only show up at runtime as `AttributeError`s.

This is worth highlighting for three reasons:

1. **Use of development tools:** In practice, you should take advantage of modern IDEs and their static analysis capabilities. Tools like Pylance/Pyright can detect mismatches between your type hints and actual usage, helping you catch bugs early before running the code.

2. **Design mindset:** More importantly, the very act of thinking in terms of interfaces trains you to better understand and structure your codebase. It helps you reason about which components should interact, what each component needs to know, and what should remain decoupled. This way of thinking is foundational to writing maintainable, extensible software, even if Python doesn't enforce it for you.

3. **Transferable concepts:** In other languages like Java, C#, or TypeScript, or Rust, interfaces (or their equivalents like Rust's traits) are part of the language itself and are enforced at compile-time. Understanding how interfaces work, and how to think in terms of contracts and separation of concerns, will serve you well across different programming environments and languages.

It’s worth clarifying that in Python, using the `Protocol` class is a way to define what’s called an **implicit interface**.

> *Definition*: An **implicit interface** specifies the expected attributes and methods of an object without requiring formal inheritance. Objects conform to the interface as long as they have the required structure, enabling flexible, loosely coupled code.

Implicit interfaces fit naturally with Python’s "duck typing" style: if it walks like a duck and quacks like a duck, it’s a duck. Protocols become especially useful when combined with tools like Pyright or Pylance, which can catch mistakes early by verifying that your code matches the interface. Next, we’ll explore how inheritance relates to interface design.

## What about inheritance?

You may be thinking, "Aha! We’ve already covered object-oriented programming in the previous lecture, and now we’re discussing interfaces&mdash;yet there’s been (almost) no mention of inheritance." That’s absolutely right, and deliberately so. As we’ll see later, inheritance can be powerful when used wisely and sparingly. But overusing it often leads to complexity and maintenance headaches. We’ll touch on these trade-offs throughout the course.

> *Definition*: **Inheritance** is a feature in object-oriented programming that allows one class (the child or subclass) to reuse and extend the behaviour of another class (the parent or superclass). It’s a way to share code between related types by building a hierarchy.

Historically, inheritance was a foundational concept in object-oriented programming, intended to promote code reuse and shared behaviour. However, over time, developers have encountered practical challenges&mdash;such as deep inheritance chains and tightly coupled designs&mdash;that have made inheritance harder to manage in large systems. As a result, some modern languages like Rust deliberately avoid or restrict traditional class-based inheritance in favor of composition and trait-based approaches. We'll explore these ideas and their relevance to scientific software as we go.

> *Definition*: An **explicit interface** requires classes to formally declare that they implement the interface, usually by inheriting from it directly.

Explicit interfaces make the connection more obvious in the code and can help prevent accidental mismatches. While implicit interfaces rely on structural compatibility ("if it has the right methods, it's valid"), explicit interfaces enforce a contract: a class must explicitly declare that it satisfies the expected behaviour. 

To complement the *implicit* `TalkingAnimalProtocol` interface we introduced earlier, we'll now define an *explicit interface* for `TalkingAnimal`&mdash;a formal declaration of the behaviour we expect from any class that wants to be treated as one. Crucially, when we implement *concrete* classes, they will inherit from this interface to make that relationship clear and enforceable.

### Explicit interface

So without further ado, let’s build our explicit `TalkingAnimal` interface.

In [ ]:
from abc import ABC, abstractmethod

The `ABC` base class tells Python this is an abstract base class, whilst `@abstractmethod` marks methods that subclasses must implement. Together, they enforce a method contract in subclasses, ensuring consistent behaviour while still allowing each subclass to provide its own implementation.

In [ ]:
class TalkingAnimal(ABC):
    """A talk animal base class

    Args:
        name: Name

    Attributes:
        name: Name
    """

    def __init__(self, name: str):
        self.name: str = name

    @abstractmethod
    def say_hello(self, friend: str) -> str:
        """Say hello to a friend

        Args:
            Name of the friend

        Returns:
            A hello message
        """

This class works much like the `Dog` class from Lecture 1, but there’s one key difference: the `@abstractmethod` decorator marks `say_hello()` as a required method. This means `TalkingAnimal` cannot be used directly, and any subclass must define its own version of `say_hello()` before you can create an instance. This helps ensure that every talking animal has a way to “speak,” while still allowing each subclass to decide what that means.

Trying to instantiate directly will fail. The `try`-`except` block here is just to catch the error so that the rest of the notebook can continue running:

In [ ]:
try:
    animal = TalkingAnimal("Charlie")  # type: ignore
except TypeError as e:
    print(f"{type(e).__name__}: {e}")

### Concrete classes

> *Definition*: A **concrete class** is a class that can be instantiated to create objects. It provides full implementations of all methods required by any interfaces or abstract base classes it inherits from, fulfilling the contract defined by those interfaces.

To create a class we can instantiate, we start by subclassing (i.e., *inheriting from*) the explicit interface. The explicit interface is also called an *abstract base class*, and the class we inherit from is written in the parentheses after the class name; this is the “explicit” part. We then provide a *concrete implementation* of the required `say_hello()` method. In the example below, we define a `Dog` class using our explicit interface. Note that this `Dog` class is a slightly different definition from the one in the first lecture because this class does not have an `age` or `species` attribute:

In [ ]:
class Dog(TalkingAnimal):
    """A dog"""

    def say_hello(self, friend: str) -> str:
        """Say hello to a friend"""
        return f"{self.name} the dog says: Woof! Hello {friend}"

**Under the hood, what's happening here?**

We've defined a blueprint for a `TalkingAnimal` that requires only one method, `say_hello`, to be implemented.  The `Dog` class provides this method, making it a *concrete* class because it fully satisfies the interface's requirements with its own logic. Since `Dog` inherits from `TalkingAnimal`, it automatically gets all the methods and attributes defined in the base class&mdash;like the `__init__` method&mdash;so there's no need to redefine them unless we want to customise their behaviour. That's why we don't have to re-implement `__init__` in `Dog`. There's clear power in having this blueprint established; we can inherit the base class's behaviour whilst defining a different `say_hello` method tailored to each animal. Let's now define a `Marmot` class as another example:

In [ ]:
class Marmot(TalkingAnimal):
    """A marmot"""

    def say_hello(self, friend: str) -> str:
        return f"{self.name} the marmot says: Squeak! Hello {friend}"

One of the key reasons classes are so powerful is because they help us follow the **DRY principle**

> *Definition*: **DRY (Don't Repeat Yourself)** means avoiding code duplication by centralizing each piece of logic in a single place.

By using inheritance, we can avoid duplicating code that’s common to many classes. For example, all talking animals share the same way to store their name and potentially other shared behaviour defined in the base class. Instead of rewriting this in every subclass, we write it once in the base class and simply inherit it. Besides reducing code duplication, classes also provide other advantages like encapsulation, which helps organise and protect data and behaviour, as we discussed in the first lecture.

## Implicit or explicit?

The natural question now is: **should I use implicit or explicit interfaces?** In practice, you’ll likely use both, depending on the situation. Although it might seem simpler to pick just one for elegance or consistency, each approach has strengths in different contexts.

**Explicit interfaces** are great for well-defined entities that fit naturally into a class hierarchy&mdash;especially when there’s significant shared behaviour and you want to create many instances. However, because explicit interfaces rely on inheritance, you must subclass and import the base class wherever you create subclasses. This can lead to a tightly coupled system and sometimes circular import issues due to the constant dependency on the base class.

On the other hand, **implicit interfaces** encourage decoupling since no inheritance or base-class imports are needed. You can define as many implicit interfaces as you want, each tailored to a specific function or application, focusing only on the behaviours needed for that case. Implicit interfaces don’t require a full behavioural specification, unlike explicit interfaces that define the entire contract.

As with many aspects of software development, there’s no single “right” solution. Thoughtful use of both approaches in appropriate contexts often yields the best results by leveraging the strengths of each where they fit best.

**Finally, there’s nothing stopping you from combining these approaches:** you can create an explicit interface to enforce core behaviour and then define multiple implicit interfaces to use throughout your broader codebase, tailoring the interface to each specific need. That is exactly what I do in my projects. This follows the model above: we can define an explicit `TalkingAnimal` interface, as well as a `TalkingAnimalProtocol` for use elsewhere in our code, especially when only certain methods are required.

**Finally, a note on learning**: This has been a dense chapter with a lot of likely new concepts, but understanding interfaces is absolutely foundational to effective software development. Both implicit and explicit interfaces underpin modularity, maintainability, and flexibility in your code. It’s normal if it feels heavy at first&mdash;returning to this chapter often, experimenting with your own interfaces, and applying the concepts in small projects will help solidify your understanding. Over time, designing and using interfaces will become second nature.

## Exercises

Using the `TalkingAnimalProtocol` and `TalkingAnimal` classes:

1. Experiment with the `TalkingAnimalProtocol` and `TalkingAnimal` classes:

    - Add new methods to the interface(s) and implement them in a concrete class (e.g. ``class Leopard``).
    - Test the flexibility of implicit interfaces: create one animal class that adheres to a single implicit interface, and another that implements multiple interfaces.
    - Mix-and-match several interfaces to create groups of animals that share similar characteristics.

    > **Tip:** For instance, a `Leopard` could  adhere to both `TalkingAnimalProtocol` and a newly defined `WalkingAnimalProtocol`, whereas `Fish` might only adhere to `TalkingAnimalProtocol`.

Apply to your own projects:

2. Identify which components could benefit from explicit interfaces and which might be better served with implicit interfaces?

3. Sketch potential interfaces for one of your projects and plan how concrete classes would implement them.